In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score
from sklearn.metrics import mean_absolute_error as mae

In [2]:
df = pd.read_csv('P:/final_all_customer_table.csv')

In [3]:
df.columns

Index(['insd_usermbr_no', 'mno_cd', 'first_prchs_dt', 'last_prchs_dt',
       'sex_clsf_cd', 'age_cd', 'total_prchs_cnt', 'weekend_prchs_cnt',
       'total_payment_amt', 'total_cpt_amt', 'total_prod_amt', 'pref_category',
       'pref_dtl_category', 'pref_tm', 'dly_prchs_amt_max',
       'dly_prchs_amt_min', 'mly_prchs_amt_min', 'mly_prchs_amt_max',
       'mly_prchs_amt_mean', 'wly_prchs_amt_max', 'wly_prchs_amt_min',
       'wly_prchs_amt_mean', 'frequency', 'recency', 'T', 'total_buys',
       'dt_step_min', 'dt_step_max', 'dt_step_mean'],
      dtype='object')

In [5]:
df.head(3)

,insd_usermbr_no,mno_cd,first_prchs_dt,last_prchs_dt,sex_clsf_cd,age_cd,total_prchs_cnt,weekend_prchs_cnt,total_payment_amt,total_cpt_amt,...,wly_prchs_amt_max,wly_prchs_amt_min,wly_prchs_amt_mean,frequency,recency,T,total_buys,dt_step_min,dt_step_max,dt_step_mean
0,IF14210296502009062300092258,US001201,2020-03-29,2020-06-29,M,042,12,3,93630,5670,...,55670,10100,23407.500000,3.0,92.0,124.0,0.0,15.0,45.0,23.0
1,IF142158893220090701150416,US001201,2020-04-11,2020-06-24,F,040,285,127,502,10968,...,200,0,41.833333,56.0,74.0,111.0,0.0,1.0,6.0,1.3
2,IF142158896620090701190525,US001201,2020-07-06,2020-07-06,M,046,1,0,8860,1500,...,8860,8860,8860.000000,0.0,0.0,25.0,0.0,0.0,0.0,0.0


**성별, 나이를 알수 없는 고객들을 따로 분리**

In [6]:
df = df[(df['age_cd'] != 'ZZZ') & (df['sex_clsf_cd']!='Z')]

**예측을 위해 나이와 성별을 따로 빼냄**

In [7]:
df_age = df[['age_cd']].astype(int)
df_sex = df[['sex_clsf_cd']]

#필요없는 컬럼 제거
df.drop(['insd_usermbr_no','first_prchs_dt','last_prchs_dt','total_buys','pref_dtl_category'], axis = 1 ,inplace = True)

## 나이 예측하기

In [8]:
age_test = df.drop(['age_cd'] ,axis = 1)

In [9]:
#시간대 라벨인코딩
le = LabelEncoder()
le.fit(age_test['pref_tm'])
age_test['pref_tm'] = le.transform(age_test['pref_tm'])

In [10]:
#카테고리 컬럼 onehot인코딩
age_test = pd.get_dummies(age_test,columns = ['sex_clsf_cd','pref_category','mno_cd'])

In [12]:
age_test.head(3)

,total_prchs_cnt,weekend_prchs_cnt,total_payment_amt,total_cpt_amt,total_prod_amt,pref_tm,dly_prchs_amt_max,dly_prchs_amt_min,mly_prchs_amt_min,mly_prchs_amt_max,...,pref_category_웹툰,pref_category_정액제,mno_cd_US001201,mno_cd_US001202,mno_cd_US001203,mno_cd_US001205,mno_cd_US001206,mno_cd_US001208,mno_cd_US001209,mno_cd_US001210
0,12,3,93630,5670,99300,22,55670,10100,15660,55670,...,0,0,1,0,0,0,0,0,0,0
1,285,127,502,10968,29660,0,100,0,0,407,...,0,0,1,0,0,0,0,0,0,0
2,1,0,8860,1500,10360,8,8860,8860,8860,8860,...,0,0,1,0,0,0,0,0,0,0


In [13]:
x_train, x_val, y_train, y_val = train_test_split(age_test, df_age, test_size = 0.25, random_state =  2020)

In [ ]:
cat =  CatBoostRegressor(depth = 10, iterations= 1000 ,eval_metric='MAE', learning_rate=0.01)
cat.fit(x_train , y_train)

In [20]:
age_pred = cat.predict(x_val)
display(mae(y_val, age_pred))

6.78090460590204

**7세 정도의 오차**  
*나이를 범주하 하여 예측하면 더 나을까??*

## 성별 예측해보기

In [129]:
df['age_cd'] = df['age_cd'].astype(int)

In [134]:
#성별 예측을 위한 라벨인코딩
df_sex = le.fit_transform(df_sex['sex_clsf_cd'])
df_sex

array([1, 0, 1, ..., 0, 0, 0])

In [135]:
df.drop(['insd_usermbr_no','first_prchs_dt','last_prchs_dt','total_buys','pref_dtl_category'], axis = 1 ,inplace = True)

In [136]:
sex_test = df.drop(['sex_clsf_cd'] ,axis = 1)

In [137]:
le = LabelEncoder()
le.fit(sex_test['pref_tm'])
#시간대 라벨인코딩
sex_test['pref_tm'] = le.transform(sex_test['pref_tm'])

In [138]:
#카테고리 컬럼 onehot 인코딩
sex_test = pd.get_dummies(sex_test,columns = ['pref_category','mno_cd'])

In [139]:
x_train, x_val, y_train, y_val = train_test_split(sex_test, df_sex, test_size = 0.25, random_state =  2020)

In [140]:
from catboost import CatBoostClassifier , Pool

In [ ]:
cat =  CatBoostClassifier(depth = 10, iterations= 1000 ,eval_metric='Accuracy', learning_rate=0.01)
cat.fit(x_train , y_train)

In [142]:
val_pred = cat.predict(x_val)
train_pred = cat.predict(x_train)

In [143]:
display(accuracy_score(y_val, val_pred), recall_score(y_val,val_pred),precision_score(y_val,val_pred),f1_score(y_val,val_pred))

0.7461741093828399

0.7301547045396906

0.7500325648039599

0.7399601619225085

In [144]:
from sklearn.metrics import confusion_matrix

In [145]:
confusion_matrix(y_val, val_pred)

array([[12278,  3838],
       [ 4256, 11516]], dtype=int64)

### 성별, 나이가 없는 고객과 있는 고객 비교

In [21]:
import sweetviz as sv

In [ ]:
df= pd.read_csv('P:/final_prchs_data.csv')
zzz = df[(df['age_cd'] == 'ZZZ') | (df['sex_clsf_cd']=='Z')]
nonzzz = df[(df['age_cd'] != 'ZZZ') & (df['sex_clsf_cd'] != 'Z')]

In [ ]:
#sweetviz의 compare를 사용해서 차이를 확인
df1 = sv.compare(zzz, nonzzz)
df1.show_html('P://sweetviz_Compare.html')

**예측 성능도 좋지 않고, 성별과 나이가 없는 고객들만의 특징도 식별할 수 없어  
전체데이터에서 제외하기로 결정**